In [215]:
import numpy as np
from sklearn import linear_model
import pandas as pd
from sklearn import preprocessing
import datetime
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
%matplotlib inline

In [159]:
save_dir='../primary_data/'

In [182]:
dems = pd.read_csv('/home/jasonmellone/Documents/nyu/DSQF/stitched_data/county_dems.csv')
dems.primary_date = pd.to_datetime(dems.primary_date)
dems = dems.dropna()

In [183]:
reps = pd.read_csv('/home/jasonmellone/Documents/nyu/DSQF/stitched_data/county_reps.csv')
reps.primary_date = pd.to_datetime(dems.primary_date)
reps=reps.dropna()

In [184]:
dem_encoding = preprocessing.LabelEncoder()

In [185]:
dem_encoding.fit(list(set(dems['1st'].unique()).union(dems['2nd'].unique()).union(dems['3rd'].unique())))

LabelEncoder()

In [186]:
dems['1st_encoded'] = dem_encoding.transform(dems['1st'].values)

In [187]:
dems['2nd_encoded'] = dem_encoding.transform(dems['2nd'].values)

In [214]:
for c in dems.columns:
    if dems[c].dtype=='object' and c not in ['1st','2nd','3rd'] and c!='party3':
        print(c)
        le = preprocessing.LabelEncoder()
        le.fit(dems[c].unique())
        dems[c] = le.transform(dems[c].values)


In [231]:
pipe = Pipeline([
    ('classify', linear_model.SGDClassifier())
])


In [236]:
param_grid = {
    
    'classify__loss':['hinge'],
    'classify__penalty':['l2', 'l1','elasticnet'],
    'classify__alpha':[0.0001,0.0001*10,0.0001*100.],
    'classify__l1_ratio':[.15,.3,.5,.7],
    'classify__learning_rate':['invscaling','optimal'],
    'classify__eta0':[0.001,.1,.5,10,100],
}

In [237]:
grid = GridSearchCV(pipe, cv=3, n_jobs=1, param_grid=param_grid)

<h1>Dems - Hinge</h1>

In [238]:
loss = 'log'
clf_first = linear_model.SGDClassifier(loss=loss)
clf_second = linear_model.SGDClassifier(loss=loss)
Xs = ['votes1','votes2','votes3','pct1','pct2','pct3','primary_type','edge1','edge2','edge3',
      'Abbreviation','area_name','state_abbreviation','PST045214','PST040210','PST120214',
      'POP010210','AGE135214','AGE295214','AGE775214','SEX255214','RHI125214','RHI225214',
      'RHI325214','RHI425214','RHI525214','RHI625214','RHI725214','RHI825214','POP715213',
      'POP645213','POP815213','EDU635213','EDU685213','VET605213','LFE305213','HSG010214',
      'HSG445213','HSG096213','HSG495213','HSD410213','HSD310213','INC910213','INC110213',
      'PVY020213','BZA010213','BZA110213','BZA115213','NES010213','SBO001207','SBO315207',
      'SBO115207','SBO215207','SBO515207','SBO415207','SBO015207','MAN450207','WTN220207',
      'RTN130207','RTN131207','AFN120207','BPS030214','LND110210','POP060210']

y_first = '1st_encoded'
y_second = '2nd_encoded'
sorted_dates = sorted(dems.primary_date.unique())

for dt in sorted_dates:
    if not dt==np.datetime64('NaT') and dt>sorted_dates[1] :
        
        dems_past = dems[dems.primary_date<dt].copy()
        dems_current = dems[dems.primary_date==dt].copy()
        
        X_train = dems_past[Xs].values
        y_train_first = dems_past[y_first].values
        y_train_second = dems_past[y_second].values
        
        X_test = dems_current[Xs].values
        y_test_first = dems_current[y_first].values
        y_test_second = dems_current[y_second].values
        
        clf_first.fit(X_train,y_train_first)
        res_first = clf_first.decision_function(X_test)

        
        clf_second.fit(X_train,y_train_second)
        res_second = clf_first.decision_function(X_test)
        
        
        break

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [239]:
grid.fit(X_train,y_train_first)

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

...

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('classify', SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'classify__loss': ['hinge'], 'classify__learning_rate': ['invscaling', 'optimal'], 'classify__l1_ratio': [0.15, 0.3, 0.5, 0.7], 'classify__alpha': [0.0001, 0.001, 0.01], 'classify__penalty': ['l2', 'l1', 'elasticnet'], 'classify__eta0': [0.001, 0.1, 0.5, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [240]:
grid.predict(X_test)

array([0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0])

In [205]:
precision, recall, _ = precision_recall_curve(y_test_first, res_first)

ValueError: Data is not binary and pos_label is not specified

In [208]:
y_test_first

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2])

In [209]:
res_first

array([ -2.74750410e+13,  -7.03295758e+12,  -1.07973565e+15,
        -1.19398384e+13,  -3.09411754e+13,   2.41579356e+12,
         3.07192261e+12,  -9.95309574e+12,   1.20293442e+12,
         2.60082344e+12,  -2.38852413e+12,   2.76003290e+12,
        -1.41120842e+13,   2.11782401e+12,   4.60012285e+12,
        -2.63101806e+14,  -4.54010394e+11])

In [193]:
dem_encoding.inverse_transform(res_first)

array(['B. SANDERS', 'B. SANDERS', 'B. SANDERS', 'B. SANDERS',
       'B. SANDERS', 'H. CLINTON', 'H. CLINTON', 'B. SANDERS',
       'H. CLINTON', 'H. CLINTON', 'B. SANDERS', 'H. CLINTON',
       'B. SANDERS', 'H. CLINTON', 'H. CLINTON', 'B. SANDERS', 'B. SANDERS'],
      dtype='<U15')

In [194]:
res_second.

numpy.datetime64('2016-02-20T00:00:00.000000000')